### Importing all the images

In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from tqdm import tqdm

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

In [2]:
class ImageDataset(Dataset) :

    def __init__(self) :
        self.root_path = 'PACS/kfold/'

        # Listing the domains
        self.domains = os.listdir(self.root_path)

        # Listing the classes 
        self.classes = os.listdir(self.root_path+'cartoon')

        self.images = []
        self.domains_y = []
        self.classes_y = []

        for i_dom, domain in tqdm(enumerate(self.domains)) :
            for i_cla, cla in enumerate(self.classes) :
                for image in os.listdir(self.root_path+domain+'/'+cla) :
                    # Finding image path
                    image_path = self.root_path+domain+'/'+cla+'/'+image
                    img = cv2.imread(image_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    self.images.append(img)
                    self.domains_y.append(i_dom)
                    self.classes_y.append(i_cla)

        self.images = np.array(self.images)
        self.domains_y = np.array(self.domains_y)
        self.classes_y = np.array(self.classes_y)


        self.images = torch.Tensor(self.images)
        self.domains_y = torch.Tensor(self.domains_y)
        self.classes_y = torch.Tensor(self.classes_y)

    def __getitem__(self, index) :
        return self.images[index], self.domains_y[index], self.classes_y[index]

    def __len__(self) :
        return self.length

In [3]:
dataset = ImageDataset()

4it [00:08,  2.22s/it]


In [4]:
img, domain, cla = dataset[0]

### Train-Test split

In [6]:
train_images = images[:1200]
test_images = images[1200:]

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
image_shape = (227, 227, 3)

# Defining the model
model = nn.Sequential(
    # Input size 3 x 227 x 227
    nn.Conv2d(3, 32, 4, 2),
    nn.ReLU(),
    # Current size 32 x 112 x 112 
    nn.Conv2d(32, 32, 4, 4),
    nn.ReLU(),
    # Current size 32 x 28 x 28 
    nn.MaxPool2d(2, 2),
    # Current size 32 x 14 x 14
    nn.Flatten(),
    nn.Linear(32*14*14, 7),
    nn.Sigmoid()
)

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()